In [1]:
import pandas as pd

In [2]:
#df = pd.read_parquet('c:/fomo_data/comments_added.parquet')

In [2]:
events = pd.read_csv('c:/fomo_data/events.csv')

In [3]:
events.head()

,Event Type,Organization Name,Facility Name,Direction,City,County,State,Create Time,Close Time,Event Description,Responding Organization Id,Latitude,Longitude
0,football game,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,12/13/2025 03:00:00 PM,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
1,concert,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,08/04/2023 18:00,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
2,concert,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,07/29/2023 07:00:00 PM,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
3,concert,Meadowlands Complex,MetLife Stadium,NaN,at East Rutherford,Bergen,NJ,07/06/2023 19:30,NaN,NaN,Meadowlands Complex,40.813460,-74.074287
4,fair,Meadowlands Complex,Meadowlands Sports Complex,NaN,East Rutherford,Bergen,NJ,06/22/2023 12:00:00 PM,NaN,NaN,Meadowlands Complex,40.814458,-74.071969


In [4]:
events.drop(columns=['Organization Name', 'Direction', 'City', 'County', 'State', 'Close Time', 'Event Description', 'Responding Organization Id'], inplace=True)

In [5]:
events.head()

,Event Type,Facility Name,Create Time,Latitude,Longitude
0,football game,MetLife Stadium,12/13/2025 03:00:00 PM,40.813460,-74.074287
1,concert,MetLife Stadium,08/04/2023 18:00,40.813460,-74.074287
2,concert,MetLife Stadium,07/29/2023 07:00:00 PM,40.813460,-74.074287
3,concert,MetLife Stadium,07/06/2023 19:30,40.813460,-74.074287
4,fair,Meadowlands Sports Complex,06/22/2023 12:00:00 PM,40.814458,-74.071969


In [6]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13377 entries, 0 to 13376
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Event Type     13377 non-null  object 
 1   Facility Name  13377 non-null  object 
 2   Create Time    13377 non-null  object 
 3   Latitude       13377 non-null  float64
 4   Longitude      13377 non-null  float64
dtypes: float64(2), object(3)
memory usage: 522.7+ KB


In [7]:
events['Event Type'] = events['Event Type'].astype('category')
events['Facility Name'] = events['Facility Name'].astype('category')

In [8]:
events['Create Time'] = events['Create Time'].astype('category')

In [9]:
events['Create Time'] = pd.to_datetime(events['Create Time'])

C:\Users\Kevin\AppData\Local\Temp\ipykernel_11844\152164604.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['Create Time'] = pd.to_datetime(events['Create Time'])


In [10]:
events.head()

,Event Type,Facility Name,Create Time,Latitude,Longitude
0,football game,MetLife Stadium,2025-12-13 15:00:00,40.813460,-74.074287
1,concert,MetLife Stadium,2023-08-04 18:00:00,40.813460,-74.074287
2,concert,MetLife Stadium,2023-07-29 19:00:00,40.813460,-74.074287
3,concert,MetLife Stadium,2023-07-06 19:30:00,40.813460,-74.074287
4,fair,Meadowlands Sports Complex,2023-06-22 12:00:00,40.814458,-74.071969


In [11]:
events['year'] = events['Create Time'].dt.year
events['month'] = events['Create Time'].dt.month
events['day'] = events['Create Time'].dt.day
events.drop(columns=['Create Time'],inplace=True)

In [12]:
events.rename(columns={'Longitude': 'longitude'}, inplace=True)
events.rename(columns={'Latitude': 'latitude'}, inplace=True)
events.rename(columns={'day_of_week': 'day'}, inplace=True)

In [13]:
events.head()

,Event Type,Facility Name,latitude,longitude,year,month,day
0,football game,MetLife Stadium,40.813460,-74.074287,2025,12,13
1,concert,MetLife Stadium,40.813460,-74.074287,2023,8,4
2,concert,MetLife Stadium,40.813460,-74.074287,2023,7,29
3,concert,MetLife Stadium,40.813460,-74.074287,2023,7,6
4,fair,Meadowlands Sports Complex,40.814458,-74.071969,2023,6,22


In [14]:
import geopandas as gpd

#sahpe file for the taxi zones
taxi_zones = gpd.read_file('c:/fomo_data/taxi.shp')

# Convert subway latitude and longitude to a GeoDataFrame
geometry = gpd.points_from_xy(events['longitude'], events['latitude'])
event_geo = gpd.GeoDataFrame(events, geometry=geometry, crs="EPSG:4326")

# Reproject taxi_zones to the same CRS as subway_geo if needed
taxi_zones = taxi_zones.to_crs(event_geo.crs)

# Perform a spatial join to find the taxi zone for each subway station using 'predicate' instead of 'op'
events_geo = gpd.sjoin(event_geo, taxi_zones, how='left', predicate='intersects')

In [15]:
events['location_id'] = events_geo['location_i']

In [16]:
events.head()

,Event Type,Facility Name,latitude,longitude,year,month,day,location_id
0,football game,MetLife Stadium,40.813460,-74.074287,2025,12,13,NaN
1,concert,MetLife Stadium,40.813460,-74.074287,2023,8,4,NaN
2,concert,MetLife Stadium,40.813460,-74.074287,2023,7,29,NaN
3,concert,MetLife Stadium,40.813460,-74.074287,2023,7,6,NaN
4,fair,Meadowlands Sports Complex,40.814458,-74.071969,2023,6,22,NaN


In [19]:
nan_mask = events.isna()
nan_counts = nan_mask.sum()
print(nan_counts)

Event Type       0
Facility Name    0
latitude         0
longitude        0
year             0
month            0
day              0
location_id      0
dtype: int64


In [18]:
events = events.dropna(subset=['location_id'])

In [20]:
index = pd.read_parquet('c:/fomo_data2/comments_merged_fixed.parquet')

In [21]:
print(len(index))
index= index.dropna()
print(len(index))

3856928
3222682


In [22]:
nan_mask = index.isna()
nan_counts = nan_mask.sum()
print(nan_counts)

locationid               0
ridership                0
year                     0
month                    0
day                      0
hour                     0
temperature_2m (°C)      0
windspeed_10m (km/h)     0
winddirection_10m (°)    0
sentiment                0
dtype: int64


In [23]:
year_counts = index['year'].value_counts()

# Display the counts
print(year_counts)

year
2023    1713672
2022    1509010
Name: count, dtype: int64


In [24]:
events.head()

,Event Type,Facility Name,latitude,longitude,year,month,day,location_id
9,concert,Madison Square Garden,40.750020,-73.992190,2023,4,29,186.0
11,concert,Radio City Music Hall,40.759902,-73.980393,2023,4,28,161.0
12,concert,Madison Square Garden,40.750020,-73.992190,2023,4,28,186.0
14,kids show/performance,Madison Square Garden,40.750020,-73.992190,2023,4,22,186.0
15,show,The Beacon Theatre,40.780449,-73.981383,2023,4,16,239.0


In [25]:
events.drop(columns=['Event Type', 'Facility Name', 'latitude', 'longitude'], inplace= True)

In [26]:
events.head()

,year,month,day,location_id
9,2023,4,29,186.0
11,2023,4,28,161.0
12,2023,4,28,186.0
14,2023,4,22,186.0
15,2023,4,16,239.0


In [ ]:
year_counts = events['year'].value_counts()
print(len(events))
# Display the counts
print(year_counts)

In [48]:
df = pd.read_parquet('c:/fomo_data2/fixed_with_comments.parquet')

In [49]:
df.head()

,locationid,ridership,year,month,day,hour,temperature_2m (°C),windspeed_10m (km/h),winddirection_10m (°),sentiment
0,2.0,1.0,2022,2,6,15,-5.6,6.600000,331.0,0.0
1,2.0,1.0,2022,2,16,23,5.4,12.100000,280.0,0.0
2,2.0,1.0,2022,2,23,8,13.5,16.299999,227.0,0.0
3,2.0,4.0,2022,2,23,19,17.5,21.600000,184.0,0.0
4,2.0,1.0,2022,3,22,10,7.4,10.800000,127.0,0.0


In [44]:
events.head()

,month,day,location_id
9,4,29,186.0
11,4,28,161.0
12,4,28,186.0
14,4,22,186.0
15,4,16,239.0


In [45]:
events.describe()

,month,day,location_id
count,10528.000000,10528.000000,10528.000000
mean,7.321429,16.070954,185.090805
std,3.764088,8.768071,36.714867
min,1.000000,1.000000,93.000000
25%,4.000000,9.000000,170.000000
50%,8.000000,16.000000,186.000000
75%,11.000000,24.000000,186.000000
max,12.000000,31.000000,247.000000


In [47]:
events.rename(columns={ 'location_id' : 'locationid'}, inplace=True )

In [50]:

# Group by 'month', 'day', 'location_id'
events_grouped = events.groupby(['month', 'day', 'locationid']).size().reset_index().drop(columns=[0])

# Perform merge
merged_df = pd.merge(df, events_grouped, on=['month', 'day', 'locationid'], how='outer', indicator=True)

# Create the 'events' column where 1 indicates a match, and 0 indicates no match
merged_df['events'] = merged_df['_merge'].apply(lambda x: 1 if x == 'both' else 0)

# Drop the '_merge' column as it's no longer needed
merged_df = merged_df.drop(columns=['_merge'])


In [52]:
merged_df.head()
print(len(merged_df))

3222684


In [55]:
print(merged_df['events'].value_counts())

events
0    3129623
1      93061
Name: count, dtype: int64


In [56]:
merged_df.to_parquet('c:/fomo_data2/complete_for_models.parquet')